In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')
import pandas as pd
from datetime import timedelta

In [2]:
from config import key_id, secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
api = tradeapi.REST(key_id, secret_key)

In [4]:
today = datetime.datetime.today().date() # or datetime.now to use local timezone
daystart=pd.Timestamp(year=today.year, month=today.month, day=today.day, hour=0,tz='America/New_York').isoformat()

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        
        dataframe['emaclose'] = tab.EMA(dataframe, timeperiod=4, price='close')
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14, price='close')
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['emapulse'] = ta.RSI(dataframe['emaclose'], timeperiod=3)
        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'40520.25'

In [7]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D

In [8]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-10-20'

In [10]:
#Limit sell all within a 
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currentsecond=time.localtime().tm_sec
    currenttime=currenthour*100+currentminute
    currenttimestamp=api.get_clock().timestamp
    
    if currenttime ==629 and currentsecond >=50: 
        try:
            assetlist=api.list_positions()
        except:
            print('http error')
        for asset in assetlist:
                #if asset is in list of positions AND is more than 2.5% loss
                # sell and turn that symbol to null
            try:
                trade=False
                if float(asset.unrealized_intraday_plpc) > 1:
                    symbol=asset.symbol

                    quoten=api.polygon.last_quote(symbol)
                    buyprice=quoten.askprice

                    #openPo/sition = api.get_position(asset.symbol)
                    #asksize,bidsize,avgbidprice,avgaskprice=quotelister(quotes)
                    returned = api.submit_order(symbol,int(asset.qty),"sell","limit","day",limit_price=buyprice-.13) # Market order to fully close position
                    print('Sell '+symbol)
            except:
                print('error')

    time.sleep(1)



KeyboardInterrupt: 

In [ ]:
goodsells